## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient
import os 

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\serviceAccount.json'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'name'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'password'

MLFLOW_TRACKING_URI = "https://mlflow-server-"


#### Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI

In [18]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [19]:
exp_name = "body-neg-side"
#client.create_experiment(exp_name)

In [20]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.val_acc > .25",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results = 3,
order_by = ['metrics.val_acc ASC']
)


In [21]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['val_acc']:.4f}")

#### Interacting with model registry
creat new verion of "skin-cancer-classifier"

In [22]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


run_id = "e991a02f0b0e4e53afe4e28388f80e97"
model_name = "tandam_body_neg-side_v1"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name=model_name)


Successfully registered model 'tandam_body_neg-side_v1'.
2023/05/17 15:18:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: tandam_body_neg-side_v1, version 1
Created version '1' of model 'tandam_body_neg-side_v1'.


<ModelVersion: aliases=[], creation_timestamp=1684313314055, current_stage='None', description='', last_updated_timestamp=1684313314055, name='tandam_body_neg-side_v1', run_id='e991a02f0b0e4e53afe4e28388f80e97', run_link='', source='gs://aiteam-mlflow-storage-bucket/models/3/e991a02f0b0e4e53afe4e28388f80e97/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
## check model version
latest_verions = client.get_latest_versions(name=model_name)


for version in latest_verions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [24]:
## set model to Staging
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1684313314055, current_stage='Staging', description='', last_updated_timestamp=1684313329959, name='tandam_body_neg-side_v1', run_id='e991a02f0b0e4e53afe4e28388f80e97', run_link='', source='gs://aiteam-mlflow-storage-bucket/models/3/e991a02f0b0e4e53afe4e28388f80e97/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [15]:
from datetime import datetime
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1662355041620, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2022-09-05', last_updated_timestamp=1662360064557, name='skin-cancer-classifier', run_id='04043070e0d740a4b4f4889b6147ff5a', run_link=None, source='./mlruns/1/04043070e0d740a4b4f4889b6147ff5a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

### Load model based on Stages

In [25]:
import mlflow.keras
#stage="Production"
stage="Staging"
name=model_name

model_path = f"models:/{name}/{stage}"

print(model_path)
tf_model = mlflow.keras.load_model(model_path)


models:/tandam_body_neg-side_v1/Staging
